## Calculate Action Spectra Statistics (Multiple Stim Wavelengths) for Multiple Cells


Hold the cell at -70 mV.\
(This is suitable for both neurons and HEK cells, although HEK cells will be quite happy to sit at a holding voltage of 0 mV.)

#### Protocol
Each run usually consists of a (rectangular) voltage step away from the expected reversal potential (see below) followed by a (rectangular) light stim at a particular wavelength (i.e. a particular LED). The light stim is fully contained within the voltage step (if there is a voltage step). For instance, the timings of the voltage and light stim ON and OFF steps might be: voltage-step ON at 3 sec, light-stim ON at 4 sec, light-stim OFF at 5 sec, voltage-step OFF at 6 sec. If the channel has slow kinetics (i.e. if the trace doesn't reach a baseline before voltage-OFF), then increase the time interval between light-OFF and voltage-OFF to record the full return to baseline.

In order to build up an Action Spectrum, this protocol sequence is repeated multiple times, changing the wavelength each time.

#### Why step the voltage prior to light stim?
Stepping the voltage prior to light stim is aimed at maximizing the change in current resulting from channel opening.\
Reversal potential for an *inhibitory channel* will be somewhere around -70 mV, so step the voltage to 0 mV (or thereabouts) prior to light stim.\
By contrast, reversal potential for an *excitatory channel* will be somewhere around 0 mV, so keep the voltage at -70 mV (or thereabouts) prior to light stim.

#### Calculations for both inhibitory and excitatory activity
This Jupyter notebook will calculate statistics from both inhibitory and excitatory light stim traces, based upon which direction the current trace moves after light stim. No manual adjustments are required to switch between calculating inhibitory or excitatory statistics.

### Import relevant modules

In [ ]:
import numpy as np
import pyabf
import matplotlib.pyplot as plt
import scipy
import glob2
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit

### Set overall variables
The file_trunk is used in filenames for saving dataframes and charts. Here, the format is: "experimentDateYYYYMMDD_cellType_proteinName_"

In [ ]:
file_trunk = '20220505_HEK_KCR_'

### Create list of all abf files with "step_hold" in their protocol name

In [ ]:
# generate list of files from current directory ending in ".abf"
path = 'example_abf_files/'
list_of_files = sorted(glob2.glob(path + "*.abf"))

# Create list of only those abf files that have "step_hold" in their protocol name
spectrumlist = []
for file in list_of_files:
    abf = pyabf.ABF(file)
    fullstring = abf.protocol
    substring1 = "step_hold"  # specific to inhibitory protocols (includes a +70 mV voltage step)
    substring2 = "stim1sec"   # specific to excitatory protocols (does not include a voltage step)
    if substring1 in fullstring:      # Search for substring1 in the fullstring
        spectrumlist.append(file)     # Add the filename to the list
    if substring2 in fullstring:      # Search for substring2 in the fullstring
        spectrumlist.append(file)     # Add the filename to the list
        
# Display the list
display(spectrumlist)

### Remove unwanted files from list

In [ ]:
#spectrumlist.remove(path + '20220407_0002.abf')
#spectrumlist.remove(path + '20220407_0003.abf')
#spectrumlist.remove(path + '20220407_0004.abf')
#spectrumlist.remove(path + '20220407_0005.abf')
#spectrumlist.remove(path + '20220407_0006.abf')
#spectrumlist.remove(path + '20220407_0007.abf')
#len(spectrumlist)

### Read out summary information of a single abf file

In [ ]:
file = spectrumlist[3]
abf = pyabf.ABF(file)
abf

### Define module for plotting all channels in a run

In [ ]:
def plot_multichannelabf(abf_filename, xaxis_min=0, xaxis_max=0, lightstim_starttime=10.8594, lightstim_endtime=11.8594):
    """For an abf file with multiple channels, plot each channel one on top of the other. 
    Need to manually set light-stim time boundaries because different protocols have different 
    settings for the light stim epoch."""
    
    # Load file to plot.
    abf = pyabf.ABF(abf_filename)
    
    # Voltage-step epoch: extract the start and end time.
    channelNumber = 0
    epochNumber = 4
    abf.setSweep(sweepNumber=0, channel=channelNumber)
    try:
        v1 = abf.sweepEpochs.p1s[epochNumber] * abf.dataSecPerPoint   # start of epoch, converted to time (in sec)
        v2 = abf.sweepEpochs.p2s[epochNumber] * abf.dataSecPerPoint   # end of epoch, converted to time (in sec)
    except:
        v1 = 10.359350000000001
        v2 = 12.359350000000001
        
    # Light-stim epoch: extract the start and end time.
    l1 = lightstim_starttime
    l2 = lightstim_endtime

    # Set other periods for calculations.
    b1 = l1 - 0.201 # pre-light stim baseline, start time
    b2 = l1 - 0.001 # pre-light stim baseline, end time
    k1 = l2 - 0.101 # last 100 ms of light stim, start time
    k2 = l2 - 0.001 # last 100 ms of light stim, end time
    
    # List of channels to plot.
    ycol = abf.channelList
    nSubplots = len(ycol)
    nSubplots_list = np.arange(0,len(ycol))

    # Plot the data from each sweep.
    f, axs = plt.subplots(nSubplots, 1, figsize=(8, (nSubplots*2.5)), sharey=False, sharex=True)
    for i,j in zip(ycol, nSubplots_list):
        abf.setSweep(sweepNumber=0, channel=i)
        sns.lineplot(x=abf.sweepX, y=abf.sweepY, ax=axs[j], label="Channel " + str(i))
        if i == 0:
            axs[j].set_ylabel(abf.sweepLabelC)
            axs[j].set_title(abf_filename)
        else:
            axs[j].set_ylabel(abf.sweepLabelY)
        axs[j].set_xlabel(abf.sweepLabelX)
        if xaxis_max == 0:
            axs[j].set_xlim(auto=True)
        else:
            axs[j].set_xlim(xaxis_min, xaxis_max)
        axs[j].axvspan(v1,v2, color='k', alpha=.1, label="voltage step")
        axs[j].axvspan(l1,l2, color='y', alpha=.2, label="light stim")
        axs[j].axvspan(b1,b2, color='b', alpha=.1, label="pre-light on")
        axs[j].axvspan(k1,k2, color='g', alpha=.1, label="pre-light off")
        axs[j].legend(loc="upper right")

    # Seaborn formatting
    f.tight_layout()
    sns.despine(fig=f)
    print('KEY TIME PERIODS:')
    print(f'Voltage step: {v1} to {v2} sec')
    print(f'Light stim: {l1} to {l2} sec')
    print(f'Baseline pre-light on: {b1} to {b2} sec')
    print(f'Baseline pre-light off: {k1} to {k2} sec')


### Plot an example of step_hold run
This is to check that the voltage step and light stime epochs are found correctly for this protocol, before proceeding with calculations.

In [ ]:
plot_multichannelabf(abf_filename=spectrumlist[3], xaxis_min=10, xaxis_max=13.5)

### Define modules to calculate, display and plot action spectrum statistics


In [ ]:
def monoExp(x, m, t, b):  
    """Single exponential decay function."""
    return m * np.exp(-t * x) + b
    
def lightstimstats(abf_filename, rate_hz=10000, lightstim_starttime=10.8594, lightstim_endtime=11.8594):
    """Calculate light stim statistics for a single input file."""
    # abf_filename : input file
    # rate_hz : the dataRate in Hz (points per sec)
    # dV : delta/change of the voltage (the size of the voltage step, determined by the stim protocol)
    # lightstim_starttime : manual entry of start of light-stim epoch
    # lightstim_endtime : manual entry of end of light-stim epoch
    
    # Load file to plot.
    abf = pyabf.ABF(abf_filename)
    rate_hz = abf.dataRate
    
    # This dictionary is what gets returned at the end.
    stats_dict={"Run ID":abf.abfID}
    stats_dict["Date Time"] = str(abf.abfDateTime)
    stats_dict["Protocol"] = abf.protocol
    stats_dict["Wavelength"] = abf.adcNames[2]
    
    # Voltage-step epoch: extract the start and end time.
    channelNumber = 0
    epochNumber = 4
    abf.setSweep(sweepNumber=0, channel=channelNumber)
    try:
        v1 = abf.sweepEpochs.p1s[epochNumber] * abf.dataSecPerPoint   # start of epoch, converted to time (in sec)
        v2 = abf.sweepEpochs.p2s[epochNumber] * abf.dataSecPerPoint   # end of epoch, converted to time (in sec)
    except:
        v1 = 10.359350000000001
        v2 = 12.359350000000001
    
    # Light-stim epoch: extract the start and end time.
    l1 = lightstim_starttime
    l2 = lightstim_endtime

    # Set other periods for calculations.
    b1 = l1 - 0.201   # pre-light stim baseline, start time
    b2 = l1 - 0.001   # pre-light stim baseline, end time
    k1 = l2 - 0.101   # last 100 ms of light stim, start time
    k2 = l2 - 0.001   # last 100 ms of light stim, end time
    
    # Convert timing periods to datapoints for calculations.
    l1_datapoints = int(l1 * rate_hz)   # light stim start time, in datapoints
    l2_datapoints = int(l2 * rate_hz)   # light stim end time, in datapoints
    k1_datapoints = int(k1 * rate_hz)   # last 100 ms of light stim, start time, in datapoints
    k2_datapoints = int(k2 * rate_hz)   # last 100 ms of light stim, end time, in datapoints
    v1_datapoints = int(v1 * rate_hz)   # voltage step start, in datapoints
    v2_datapoints = int(v2 * rate_hz)   # voltage step end, in datapoints
    
    # Set sweep for calculations (need y-axis to be photocurrent).
    channelNumber = 1
    abf.setSweep(sweepNumber=0, channel=channelNumber, baseline=[b1,b2]) # N.B. baseline units need to be in seconds (so don't convert to datapoints)
    
    # Check for excitatory activity.
    if np.mean(abf.sweepY[l1_datapoints:(l1_datapoints+100)]) < 0:   # if the trace goes downwards after light stim ON (i.e. excitatory channel activity)
        # then flip the y-data across the x-axis (* -1) so that the following analysis is the same
        Yl = -1 * np.array(abf.sweepY[l1_datapoints:l2_datapoints])   # N.B. abf.sweepY is in units datapoints
        Yk = -1 * np.array(abf.sweepY[k1_datapoints:k2_datapoints])
        Yp = -1 * np.array(abf.sweepY[l2_datapoints:v2_datapoints])
        excitatory = True
        dV = 0 *1e-3 # dV is in units Volts
    else:      # if the trace goes upwards after light stim ON (i.e. inhibitory channel activity)
        Yl = np.array(abf.sweepY[l1_datapoints:l2_datapoints])   # N.B. abf.sweepY is in units datapoints
        Yk = np.array(abf.sweepY[k1_datapoints:k2_datapoints])
        Yp = np.array(abf.sweepY[l2_datapoints:v2_datapoints])
        excitatory = False
        dV = 70 *1e-3 # dV is in units Volts
        
    stats_dict['Excitatory?'] = excitatory
    stats_dict["dV"] = dV*1e3   # dV is in units Volts, "*1e3" to convert to milliVolts (mV)
    
    ## PERIOD 1: DURING LIGHT-STIM (i.e. between L1 and L2)
    # Make arrays for x and y values:
    Xl = np.array(abf.sweepX[l1_datapoints:l2_datapoints])   # N.B. abf.sweepX is in units datapoints
    
    # Calculate peak current.
    peakcurrent = np.amax(Yl)*1e-12  # peakcurrent is in Amps ("*1e-12" converts pA to A)     
    stats_dict['Peak photocurrent (nA)'] = peakcurrent *1e9  # peakcurrent is in units Amps, "*1e9" to convert to nanoAmps (nA)
    
    # Calculate time-to-peak: find X (time) at max Y (photocurrent).
    X_atMaxY = Xl[np.argmax(Yl)]
    timeToPeak = (X_atMaxY - l1)   # timeToPeak is in seconds
    stats_dict['Time to peak (ms)'] = timeToPeak *1e3  # timeToPeak is in units seconds, "*1e3" to convert to milliseconds (ms)
    
    # Calculate tau-off from peak current to steady-state.
    # make new arrays starting from maximum Y value (argmax) i.e. peak photocurrent
    Yl_fromMaxY = Yl[np.argmax(Yl):]
    Xl_fromMaxY = Xl[np.argmax(Yl):]
    tau1_dict={"Xl":Xl}
    tau1_dict["Yl"] = Yl
    tau1_dict["Xl_fromMaxY"] = Xl_fromMaxY
    # Calculate the tau value and fit; perform the fit:
    try:        # placed inside try/except because sometimes a fit cannot be found in the given maxfev
        p0_l = (1000, 0.1, 1000)   # start with values near those we expect
        params_l, cv = scipy.optimize.curve_fit(monoExp, Xl_fromMaxY, Yl_fromMaxY, p0_l, maxfev=30000)
        m_l, t_l, b_l = params_l
        tau_light = (1 / t_l)    # tau_light is in units seconds
        # determine quality of the fit:
        squaredDiffs = np.square(Yl_fromMaxY - monoExp(Xl_fromMaxY, m_l, t_l, b_l))
        squaredDiffsFromMean = np.square(Yl_fromMaxY - np.mean(Yl_fromMaxY))
        rSquared_light = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)
        stats_dict["Tau-off1 (ms)"] = tau_light*1e3      # tau_light is in units seconds, "*1e3" to convert to milliSec (ms)
        stats_dict["R² for Tau-off1"] = rSquared_light
        tau1_dict["m_l"] = m_l
        tau1_dict["b_l"] = b_l
        tau1_dict["t_l"] = t_l
    except:
        pass

    
    ## PERIOD 2: Last 100 ms of LIGHT-STIM (i.e. between K1 and K2)
    # Calculate steady-state photocurrent.
    steadycurrent = np.mean(Yk)*1e-12  # find mean for the array of sweepY values between k1 and k2; steadycurrent is in Amps ("*1e-12" converts pA to A) 
    stats_dict['Steady-state photocurrent (nA)'] = steadycurrent *1e9  # steadycurrent is in units Amps, "*1e9" to convert to nanoAmps (nA)

    # Calculate ratio between steady-state and peak photocurrents
    ratio_sp = steadycurrent/peakcurrent    # no units (both steadycurrent and peakcurrent in units Amps)
    stats_dict['Ratio steady-to-peak'] = ratio_sp  # no units
    
    
    ## PERIOD 3: AFTER LIGHT-STIM  (i.e. between L2 and V2)
    # Calculate tau-off from peak current to steady-state.
    # Make arrays for x and y values:
    Xp = np.array(abf.sweepX[l2_datapoints:v2_datapoints])
    # make new arrays starting from maximum Y value (argmax) i.e. peak photocurrent
    Yp_fromMaxY = Yp[np.argmax(Yp):]
    Xp_fromMaxY = Xp[np.argmax(Yp):]
    tau2_dict={"Xp":Xp}
    tau2_dict["Yp"] = Yp
    tau2_dict["Xp_fromMaxY"] = Xp_fromMaxY
    # perform the fit
    try:      # placed inside try/except because sometimes a fit cannot be found in the given maxfev
        p0_p = (1000, 0.1, 30) # start with values near those we expect
        params_p, cv = scipy.optimize.curve_fit(monoExp, Xp_fromMaxY, Yp_fromMaxY, p0_p, maxfev=30000)
        m_p, t_p, b_p = params_p
        tau_postlight = (1 / t_p)    # tau_postlight is in units seconds
        # determine quality of the fit
        squaredDiffs = np.square(Yp_fromMaxY - monoExp(Xp_fromMaxY, m_p, t_p, b_p))
        squaredDiffsFromMean = np.square(Yp_fromMaxY - np.mean(Yp_fromMaxY))
        rSquared_postlight = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)
        stats_dict["Tau-off2 (ms)"] = tau_postlight*1e3      # tau_postlight is in units seconds, "*1e3" to convert to milliSec (ms)
        stats_dict["R² for Tau-off2"] = rSquared_postlight
        tau2_dict["m_p"] = m_p
        tau2_dict["b_p"] = b_p
        tau2_dict["t_p"] = t_p
    except:
        pass
    
    return stats_dict, tau1_dict, tau2_dict

def display_lightstimstats(stats_dict):
    """Given a lightstimstats dictionary, show the important stuff with its units."""
    print("Run ID: " + stats_dict["Run ID"])
    print("Date Time: " + stats_dict["Date Time"])
    print("Protocol: " + stats_dict["Protocol"])
    print("Wavelength: " + stats_dict["Wavelength"])
    print("Voltage step: %.02f mV"%(stats_dict["dV"]))
    print("Excitatory?: " + str(stats_dict["Excitatory?"]))
    print("Peak photocurrent: %.03f nA"%(stats_dict["Peak photocurrent (nA)"]))
    print("Time to peak: %.02f ms"%(stats_dict["Time to peak (ms)"]))
    print("Steady-state photocurrent: %.03f nA"%(stats_dict["Steady-state photocurrent (nA)"]))
    print("Ratio steady-to-peak: %.02f"%(stats_dict["Ratio steady-to-peak"]))
    try:
        print("Tau-off1 (ms): %.03f ms"%(stats_dict["Tau-off1 (ms)"]))
        print("Tau-off1 R² (peak-to-steadystate):  %.03f"%(stats_dict["R² for Tau-off1"]))
        print("Tau-off2 (ms): %.03f ms"%(stats_dict["Tau-off2 (ms)"]))
        print("Tau-off2 R² (peak-to-steadystate):  %.03f"%(stats_dict["R² for Tau-off2"]))
    except:
        print("Tau-off1 (ms): calculation error")
        print("Tau-off1 R² (peak-to-steadystate):  calculation error")
        print("Tau-off2 (ms): calculation error")
        print("Tau-off2 R² (peak-to-steadystate):  calculation error")

def plot_lightstimstats(stats_dict, tau1_dict, tau2_dict, savefig=False):
    """Plot the lightstimstats curves to show the goodness of fit."""
    fig = plt.figure(figsize=(15, 5))
    plt.plot((tau1_dict["Xl"]),(tau1_dict["Yl"]), '.', label="during light stim, data")
    plt.plot((tau2_dict["Xp"]), (tau2_dict["Yp"]), '.', label="after light stim, data")
    try:
        plt.plot((tau1_dict["Xl_fromMaxY"]), monoExp((tau1_dict["Xl_fromMaxY"]), (tau1_dict["m_l"]), (tau1_dict["t_l"]), (tau1_dict["b_l"])), '--', label="during light stim, fitted")
    except:
        pass
    try:
        plt.plot((tau2_dict["Xp_fromMaxY"]), monoExp((tau2_dict["Xp_fromMaxY"]), (tau2_dict["m_p"]), (tau2_dict["t_p"]), (tau2_dict["b_p"])), '--', label="after light stim, fitted")
    except:
        pass
    plt.ylabel("Current (pA)")
    plt.xlabel("Time (sec)")
    plt.title(stats_dict["Run ID"] + " " + stats_dict["Protocol"] + " tau-fit")
    if savefig==True:
        plt.savefig(stats_dict["Run ID"] + "_" + stats_dict["Protocol"] + "_tau-fit" +'.png')
    plt.legend()
    plt.show()
    

### Calculate light stim statistics for single run

In [ ]:
# Enter an abf file:
file = spectrumlist[3]

# Run calculations.
stats_dict, tau1_dict, tau2_dict = lightstimstats(file)

# Display key statistics.
display_lightstimstats(stats_dict)

# Plot chart of key part of curve.
plot_lightstimstats(stats_dict, tau1_dict, tau2_dict)

### Define module for calculating light stim statistics for multiple runs, putting the results into a dataframe

In [ ]:
def multirun_lightstimstats(lightstim_list, makeplots=False, savefigs=False):
    """Enter a list of lightstim files to calculate statistics for all of them."""
    
    # Create dataframe for storing output values.
    df_spectrum = pd.DataFrame()

    # Iterate across each file in the list.
    for i,j in zip(lightstim_list, np.arange(0,len(lightstim_list))):

        # Run calculations.
        stats_dict, tau1_dict, tau2_dict = lightstimstats(i)

        # Put stats_dict into a dataframe.
        df_stats_dict = pd.DataFrame([stats_dict], index=[j])
        
        # Append this dataframe to main dataframe.
        df_spectrum = pd.concat([df_spectrum,df_stats_dict], )
        
        if makeplots==True:
            # Plot chart of key part of curve.
            plot_lightstimstats(stats_dict, tau1_dict, tau2_dict, savefig=savefigs)
            
    return df_spectrum


### Calculate light stim statistics for list of two runs (small test case)

In [ ]:
# Create input list of filenames to calculate statistics for.
spectrumlist_short = [spectrumlist[3], spectrumlist[4]]

# Run module with input list.
multirun_lightstimstats(spectrumlist_short, makeplots=True)

## Calculate light stim statistics for all files in spectrumlist
Calculate light stim statistics for all files in spectrumlist and put all into a dataframe.

In [ ]:
# Run module with input list to calculate all statistics for all input files and output into a dataframe.
df_spectrum = multirun_lightstimstats(spectrumlist, makeplots=False)

In [ ]:
# Display the output dataframe.
display(df_spectrum)

### Add columns to the dataframe for cell labels (manual annotation)


In [ ]:
# Add extra columns manually (write lists, add lists to dataframe)       
cellpatchID = []  # For a particular date, what number cell does each run correspond to?
cellpatchID += 6 * ['01']
cellpatchID += 6 * ['05']  
cellpatchID += 6 * ['13']
cellpatchID += 6 * ['08']
cellpatchID += 6 * ['10']
cellpatchID += 6 * ['01']  
cellpatchID += 6 * ['02']

construct = []    # What is the protein variant's name?
construct += 1* 6 * ['mutantX']
construct += 1* 6 * ['mutantY']
construct += 1* 6 * ['mutantZ']
construct += 1* 6 * ['mutantA_excit']
construct += 1* 6 * ['mutantB_excit']
construct += 2* 6 * ['mutantQ']

# Add the lists to the dataframe as new columns
df_spectrum["Cell patch ID"] = cellpatchID
df_spectrum["Construct"] = construct

# Make a unique cell ID by combining the date with the cellpatchID => into new column "Date_Cell"
df_spectrum['Date'] = df_spectrum['Date Time'].str.replace(r'\ .*', '', regex=True).replace(r'\-', '', regex=True) # pull out date from "Date Time"
df_spectrum['Date_Cell'] = df_spectrum['Date'] + "_" + df_spectrum['Cell patch ID'].astype(str) # combine "Date" and "Cell patch ID"
df_spectrum

# Display the dataframe
display(df_spectrum)

### Plot all action spectrum statistics to compare within and between categories
The categories here are the wavelengths.

In [ ]:
# Set category column for X-axis
xcol = "Wavelength"

# Set category column for hue (the color of the dots)
zcol = 'Construct' # or 'Date_Cell'

# List of statistics (column names) to plot
ycol = ['Peak photocurrent (nA)', 
        'Steady-state photocurrent (nA)',
        'Time to peak (ms)',
        'Tau-off1 (ms)',
        'R² for Tau-off1',
        'Tau-off2 (ms)', 
        'R² for Tau-off2',
        # add more statistics' names here for more plots
       ]
nSubplots = len(ycol)
nSubplots_list = np.arange(0,len(ycol))

# Plot the data from each column in the dataframe
f, axs = plt.subplots(1, nSubplots, figsize=((nSubplots*3), 4), sharey=False)
for i,j in zip(ycol, nSubplots_list):
    sns.stripplot(data=df_spectrum, x=xcol, y=i, hue=zcol, ax=axs[j])
    axs[j].tick_params(labelrotation=45)

# Seaborn formatting
f.tight_layout()
sns.despine(fig=f)

### Save figure as a png file

In [ ]:
out_path = 'output/'
file_branch = 'actionspectra_'

In [ ]:
f.figure.savefig(out_path + file_trunk + "actionspectra_stats.png")

## Export dataframe to csv files

### Export dataframe, long-form:

In [ ]:
# Export dataframe to csv file.
df_spectrum.to_csv(out_path + file_trunk + file_branch + 'stats_longform.csv', index=False)

print('Wrote full dataframe to csv. Long-form: each row is one cell at one wavelength, each column is one statistic.')

### Export dataframe by components, wide-form:

In [ ]:
# Set up features for rearranging dataframe into pivot tables.
pivot_index = 'Wavelength'
pivot_columns = 'Date_Cell'
pivot_values_list = ['Construct', 
                     'Peak photocurrent (nA)', 
                     'Steady-state photocurrent (nA)', 
                     'Ratio steady-to-peak', 
                     'Time to peak (ms)',
                     'Tau-off1 (ms)',
                     'R² for Tau-off1',
                     'Tau-off2 (ms)',
                     'R² for Tau-off2',
                     # add more statistics' names here for more tables
              ]

# Convert to pivot table and export to csv:
for i in pivot_values_list:
    df_spectrum_wide = df_spectrum.pivot(index=pivot_index, columns=pivot_columns, values=i)
    df_spectrum_wide.to_csv(out_path + file_trunk + file_branch + i + '_stats_wideform.csv', index=True)
    
print('Wrote all pivot tables to csv. Wide-form: each table is one statistic, each row is one wavelength, each column is one cell.')